In [1]:
import os
os.chdir("..")  # Moves one directory up

In [2]:
import pandas as pd 
import numpy as np 
from utils.config import base_path
from utils.preprocess import concat_metadata_with_gene_expression,prepare_dataset_for_feauture_selection,normalize_features,train_lasso_logistic_regression,get_significant_genes,evaluate_model
from sklearn.model_selection import train_test_split


In [3]:
gene_expression = pd.read_csv(base_path/ "gene_expression.csv")
meta_data = pd.read_csv(base_path/ "meta_data.csv")

df = concat_metadata_with_gene_expression(meta_data,gene_expression,filter_nan_target=True)
df.head()

,SampleID,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at,Tissue,disease state,protocol,disease activity score (das28),Response status,Gender
0,GSM381194,2.377898,7.109049,9.630326,2.331110,2.326892,7.145603,3.134364,2.374527,9.394731,...,2.389734,2.391520,2.372542,2.383567,whole blood,rheumatoid arthritis,anti_TNF therapy,4.51,Responder,Male
1,GSM381195,2.378662,5.156438,8.871972,2.337406,2.325880,6.724430,3.132534,2.371931,8.343010,...,2.389150,2.388430,2.371702,2.383306,whole blood,rheumatoid arthritis,anti_TNF therapy,6.04,Non_responder,Female
2,GSM381196,2.382631,5.321988,8.348816,2.334704,2.326533,6.856559,2.996339,2.373904,8.742820,...,2.389496,2.390634,2.373524,2.384077,whole blood,rheumatoid arthritis,anti_TNF therapy,5.92,Non_responder,Male
4,GSM381198,2.461926,5.980662,8.807910,2.334267,2.327032,7.758407,3.444969,2.375944,9.125310,...,2.389583,2.392193,2.373700,2.384135,whole blood,rheumatoid arthritis,anti_TNF therapy,6.00,Non_responder,Female
5,GSM381199,3.290609,6.352822,9.133817,2.337572,2.328993,7.381657,3.148597,2.399196,9.176213,...,2.397028,2.400408,2.378989,2.390307,whole blood,rheumatoid arthritis,anti_TNF therapy,5.20,Responder,Female


In [4]:
# df = prepare_dataset_for_feauture_selection(df)
# df

In [5]:
print(df["Response status"].unique())


['Responder' 'Non_responder']


In [6]:
X,y = prepare_dataset_for_feauture_selection(df)
# Step 1: Split into 80% Train, 20% (Val + Test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 2: Split the remaining 20% into 10% Validation, 10% Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Step 3: Normalize features
X_train_scaled = normalize_features(X_train)
X_val_scaled = normalize_features(X_val)


# Step 5: Train Lasso Logistic Regression
lasso_model = train_lasso_logistic_regression(X_train, y_train)

# Step 6: Extract significant genes
significant_genes = get_significant_genes(lasso_model, X.columns)

# # Step 7: Evaluate Model
# evaluate_model(lasso_model, X_test, y_test)


In [7]:
significant_genes

,Gene,Coefficient
48661,239412_at,0.365614
13588,204141_at,0.267668
11357,201909_at,0.153915
14930,205483_s_at,0.126555
36729,227474_at,0.119524
54403,44673_at,0.103471
21272,211965_at,0.077807
15730,206284_x_at,0.074871
23358,214059_at,0.017045
16122,206676_at,-0.013283


In [8]:
# Select top 10 genes based on absolute coefficient values
top_10_features = significant_genes.reindex(significant_genes['Coefficient'].abs().sort_values(ascending=False).index).head(10)
top_10_features

,Gene,Coefficient
23131,213831_at,-0.366530
48661,239412_at,0.365614
13588,204141_at,0.267668
22073,212768_s_at,-0.193174
11357,201909_at,0.153915
16942,207500_at,-0.153483
36449,227194_at,-0.146002
21068,211748_x_at,-0.140918
14930,205483_s_at,0.126555
36729,227474_at,0.119524


In [9]:
list(top_10_features.Gene)

['213831_at',
 '239412_at',
 '204141_at',
 '212768_s_at',
 '201909_at',
 '207500_at',
 '227194_at',
 '211748_x_at',
 '205483_s_at',
 '227474_at']